In [1]:
%load_ext autoreload
%autoreload 2

%cd '..'

/dss/dsshome1/03/di93fup/polarization_reddit


In [2]:
import logging
# from datetime import datetime

# date = datetime.now().strftime("%m_%d_%Y")

# logging.basicConfig(
#     level=logging.INFO,
#     format="%(asctime)s [%(levelname)s] %(message)s",
#     handlers=[
#         logging.FileHandler(
#             f"data/logs/mass_shootings_filter_comments_notebook_{date}.log"
#         ),
#         logging.StreamHandler(),
#     ],
# )


In [3]:
import pandas as pd
from tqdm import tqdm
tqdm.pandas()

from load.utils import (
    load_df_from_parquet,
)
from preprocessing.utils import (
    save_event_comments,
    load_event_comments,
    build_vocab,
    save_event_vocab,
)

from preprocessing.constants import (
    MASS_SHOOTINGS_EVENTS_INFO,
    MASS_SHOOTINGS_KEYWORDS,
    MIN_OCCURENCE_FOR_VOCAB,
    get_event_regex,
)

In [4]:
YEARS = [2015, 2016, 2017, 2018]

OVERALL_EVENT_NAME = "mass_shootings"
EVENT_NAMES = list(MASS_SHOOTINGS_EVENTS_INFO.keys())

MASS_SHOOTING_EVENTS_KEYWORDS = [
    keyword
    for event_name in EVENT_NAMES
    for keyword in MASS_SHOOTINGS_EVENTS_INFO[event_name]["keywords"]
]


In [5]:
MASS_SHOOTING_REGEX = get_event_regex(
    MASS_SHOOTINGS_KEYWORDS,
    MASS_SHOOTING_EVENTS_KEYWORDS,
    operator="and",
)

logging.info(MASS_SHOOTING_REGEX)


In [6]:
MASS_SHOOTING_REGEX

'(?:shoot|gun|kil|attack|massacr|victim).*(?:charleston|chattanoog|umpqu|roseburg|colorado springs|plan par|san bernardino|kalamazoo|orlando|puls nightclub|dalla|baton roug|burlington|cascad mal|fort lauderd|fresno|up|san francisco|vega|harvest fest|mandalay bay|rout 91|thornton|walmart|denv|sutherland springs|parkland|mars stonem|nashvil|waffl hous|sant fe|annapol|capit gazet|pittsburgh|tre of lif|thousand oak)'

## Filter event comments

In [7]:
logging.info("Load partisan comments")
partisan_comments = pd.concat(
    [
        load_df_from_parquet(
            file_name=f"user_partisan_comments_{year}.parquet",
        )
        for year in YEARS
    ]
)


In [9]:
logging.info("Filtering event data based on keywords...")
event_comments = partisan_comments[
    partisan_comments["tokens"].str.contains(
        MASS_SHOOTING_REGEX,
    )
].copy()
logging.info("finished keyword filtering")


In [11]:
logging.info(event_comments.shape)
logging.info(event_comments.dtypes)

In [12]:
del partisan_comments

In [14]:
logging.info(f"Nr of event comments: {len(event_comments)}")

In [15]:
logging.info("saving event comments...")
save_event_comments(event_comments, OVERALL_EVENT_NAME)

## Build event vocabulary

In [17]:
# Read event data
# events_comments = load_event_comments(
#     OVERALL_EVENT_NAME,
# )

event_vocab = build_vocab(
    event_comments["tokens"],
    min_comment_freq=MIN_OCCURENCE_FOR_VOCAB,
)


In [ ]:
logging.info("Vocabulary length")
logging.info(len(event_vocab))

In [ ]:
logging.info("Saving event vocab")
save_event_vocab(event_vocab, OVERALL_EVENT_NAME)